In [1]:
#Notwendige Imports für das Projekt
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors

from GLC.metrics import top_30_error_rate, top_k_error_rate_from_sets, predict_top_30_set
from GLC.submission import generate_submission_file

In [2]:
DATA_PATH = Path("./data/")

In [3]:
df_obs = pd.read_csv(DATA_PATH / "observations" / "observations_fr_train.csv", sep=";", index_col="observation_id")
df_obs_test = pd.read_csv(DATA_PATH / "observations" / "observations_fr_test.csv", sep=";", index_col="observation_id")


In [4]:
obs_id_train = df_obs.index[df_obs["subset"] == "train"].values
obs_id_val = df_obs.index[df_obs["subset"] == "val"].values

## Lösung

Benutzt werden die Umweltmerkmale der Beobachtungen. Die jeweiligen Daten der Beobachtungen stehen in der `environmental_vectors.csv`. Die Beschreibung der Merkmale stehen in der `environmental_variables.csv`.

In [5]:
df_env = pd.read_csv(DATA_PATH / "pre-extracted" / "environmental_vectors.csv", sep=";", index_col="observation_id")
df_env_description = pd.read_csv(DATA_PATH / "metadata" / "environmental_variables.csv", sep=";", index_col="name")

In [6]:
#Beispiel für einige Observationen:
df_env.head()

,bio_1,bio_2,bio_3,bio_4,bio_5,bio_6,bio_7,bio_8,bio_9,bio_10,...,bio_18,bio_19,bdticm,bldfie,cecsol,clyppt,orcdrc,phihox,sltppt,sndppt
observation_id,,,,,,,,,,,,,,,,,,,,,
10000000,1.420833,6.908333,29.272598,614.14930,15.1,-8.5,23.600000,-1.000000,9.183333,9.466667,...,248.0,358.0,2082.0,988.0,29.0,13.0,63.0,62.0,34.0,53.0
10000001,8.837500,9.858334,37.771393,586.81390,23.8,-2.3,26.099998,6.016667,16.383333,16.383333,...,226.0,288.0,1816.0,1142.0,20.0,22.0,39.0,58.0,41.0,36.0
10000002,6.241667,8.350000,32.239384,632.86090,21.0,-4.9,25.900000,3.033333,14.200000,14.200000,...,268.0,317.0,1346.0,1075.0,29.0,22.0,54.0,59.0,40.0,38.0
10000003,12.554167,9.525001,40.189877,541.80865,25.9,2.2,23.699999,6.850000,19.350000,19.350000,...,157.0,257.0,1227.0,1383.0,21.0,28.0,18.0,71.0,46.0,25.0
10000004,8.029167,10.075000,36.636364,633.01750,23.7,-3.8,27.500000,4.616667,16.083334,16.083334,...,214.0,280.0,2833.0,1202.0,24.0,25.0,33.0,69.0,38.0,37.0


In [7]:
# Die entsprechenden Beschreibungen der Merkmale
pd.set_option('display.max_colwidth', None)
print(df_env_description)
pd.reset_option('display.max_colwidth')

                                                       description resolution
name                                                                         
bio_1                                      Annual Mean Temperature  30 arcsec
bio_2   Mean Diurnal Range (Mean of monthly (max temp - min temp))  30 arcsec
bio_3                          Isothermality (bio_2/bio_7) (* 100)  30 arcsec
bio_4           Temperature Seasonality (standard deviation * 100)  30 arcsec
bio_5                             Max Temperature of Warmest Month  30 arcsec
bio_6                             Min Temperature of Coldest Month  30 arcsec
bio_7                       Temperature Annual Range (bio_5-bio_6)  30 arcsec
bio_8                          Mean Temperature of Wettest Quarter  30 arcsec
bio_9                           Mean Temperature of Driest Quarter  30 arcsec
bio_10                         Mean Temperature of Warmest Quarter  30 arcsec
bio_11                         Mean Temperature of Coldest Quart

In [16]:
# Entnahme des Trainings- und Validierungsdatensatzes
X_train = df_env.loc[obs_id_train].values    
X_val = df_env.loc[obs_id_val].values

y_train = df_obs.loc[obs_id_train]["species_id"].values  #Id der Arten vom Trainingsdatensatz
y_val = df_obs.loc[obs_id_val]["species_id"].values # Id der Arten vom Validierungsdatensatz

In [17]:
# Daten enthalten NaN --> Ersetzen der NaN mithilfe von SimpleImputer

from sklearn.impute import SimpleImputer
imp = SimpleImputer(
    missing_values=np.nan,
    strategy="constant",
    fill_value=np.finfo(np.float32).min,
)
imp.fit(X_train)

X_train = imp.transform(X_train)
X_val = imp.transform(X_val)

In [14]:
# Importieren des RandomForestClassifier
# Erstellen des Forests auf Grundlage der Umweltmerkmalen der Beobachtung und der dazugehörigen Art

from sklearn.ensemble import RandomForestClassifier
est = RandomForestClassifier(n_estimators=16, max_depth=10, n_jobs=-1)
est.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, n_estimators=16, n_jobs=-1)

In [18]:
# Treffe Vorhersagen für den Validierungssatz
# Für eine Beobachtung erhält man vom Random Forest die Wahrscheinlichkeiten für jede Art (Predict class probabilities)
# Die 30 Arten mit der größten Wahrscheinlichkeiten werden als Vorhersage für eine Beobachtung ausgewählt
# Anschließend: Berechnung der Top-30 Fehlerrate

predictions = est.predict_proba(X_val)
s_pred = predict_top_30_set(predictions)
y_score = top_k_error_rate_from_sets(y_val, s_pred)
print("Top-30 Fehlerrate: {:.1%}".format(y_score))

Top-30 Fehlerrate: 80.8%
